In [1]:
from dataclasses import dataclass
import pandas as pd
import numpy as np


In [2]:



@dataclass
class Transform_tools:
    df:pd.DataFrame
    def normalize_Data(self,df:pd.DataFrame):
        """
        Нормализация и очистка данных отчета.

        Аргументы:
        df (DataFrame): DataFrame pandas с данными отчета.

        Возвращает:
        DataFrame: Очищенный и нормализированный DataFrame pandas.
        """

        #получаем список колонок с конверсиями
        conversionsCols = []

        #меняем -- на нули и переводим в инт
        for col in df.columns:
            if 'Conversions' in col:
                df[col] = df[col].replace('--','0').astype(int)
                conversionsCols.append(col)

        df.fillna('--',inplace=True)

        # создаем столбец, который будет суммировать все столбцы с conversions в названии
        df['Conversions'] = df[conversionsCols].sum(axis=1)

        # удаляем все столбцы с conversions в названии, кроме 'Conversions'
        for col in conversionsCols:
            if col != 'Conversions':
                df = df.drop(columns=[col])


            # переименовываем столбцы на русский язык
        df = df.rename(columns={
            'CampaignName': 'Название кампании',
            'AdGroupName': 'Название группы объявлений',
            'Device': 'Устройство',
            'Date': 'Дата',
            'Gender': 'Пол',
            'TargetingLocationName': 'ГЕО',
            'Impressions': 'Показы',
            'Clicks': 'Клики',
            'Cost': 'Расход',
            'Conversions': 'Конверсии'
        })

        print("Данные обработаны.")
        return df
    def calculate_metrics(self):
    """
    Обработка данных и добавление столбцов CPC, CPA и CR.

    Аргументы:
    df (DataFrame): DataFrame pandas с данными отчета.

    Возвращает:
    DataFrame: DataFrame pandas с добавленными столбцами CPC, CPA и CR.
    """
    # Вычисление CPC
    df = self.df
    df['CPC (руб.)'] = round(df['Расход'] / df['Клики'],2)
    
    # Вычисление CPA
    df['CPA (руб.)'] = round(df['Расход'] / df['Конверсии'])
    
    # Вычисление CR
    df['CR (%)'] = round((df['Конверсии'] / df['Клики']) * 100,2)
    
    # Заполнение бесконечных значений в результате деления на ноль
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    return df

In [3]:
def param_agg(df, param):
    # Расчет агрегированных значений по интересующим параметрам
    agg_values = df.groupby(param, as_index=False).agg({'Показы': 'sum', 'Клики': 'sum', 'Расход': 'sum', 'Конверсии': 'sum'})

    if param == 'Дата':
        agg_values = agg_values.sort_values('Дата')
    else:
        agg_values = agg_values.sort_values('Расход', ascending=False)

    agg_values = calculate_metrics(agg_values)  # Обновление агрегированного DataFrame с помощью функции calculate_metrics

    return agg_values